# Tuning Empirical Gaussian FLORIS Model to SCADA Using Interpolation and Mathematical Optimization

In this notebook, the Empirical Gaussian FLORIS Model (emgauss) will be tuned to align with SCADA data using an interpolation/mathematical optimization technique that determines the parameter value(s) that minimize the error (mean squared error) between SCADA and FLORIS energy ratios. 

The parameters of interest in this tuning exercise are 'wake_expansion_rates' (1st expansion rate) and 'horizontal_deflection_gain_D'. These parameters are associated with the following operating scenarios:

wake_expansion_rates => basline case

horizontal_deflection_gain_D => wake steering case

## Import Relevant Libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path

from floris.tools import FlorisInterface

from flasc.model_estimation.floris_tuner import FlorisTuner
from flasc.visualization import plot_floris_layout, plot_layout_only, plot_layout_with_waking_directions, plot_binned_mean_and_ci


In [ ]:
# Suppress warnings
import warnings

warnings.filterwarnings('ignore')

## Load and Inspect SCADA

Load pre-processed SCADA data with power curve fiiltering and northing calibration applied, and inspect the data.

In [ ]:
def load_scada(scada_path: str):
    """
    Load SCADA

    Args:
        scada_path (:py:obj:`str`): Path to load SCADA from.

    Returns:
        df_scada (:py:obj:`pd.DataFrame`): SCADA data.
    """
    
    df_scada = pd.read_feather(scada_path)

    # If 'downsample' is True, downsample SCADA data to [x] minute averages to speed things up
    # if downsample:
    #     cols_angular = [c for c in df_scada if (("wd_" in c) or ("yaw_" in c))]
    #     df_scada = fto.df_downsample(
    #         df_scada,
    #         cols_angular=cols_angular,
    #         window_width=td(seconds=600),
    #     )

    return df_scada

In [ ]:
# Specify SCADA file path and load the dataframe
scada_path = os.path.join(Path.cwd(), "postprocessed", "df_scada_data_60s_filtered_and_northing_calibrated.ftr")
df_scada = load_scada(scada_path=scada_path)

In [ ]:
# Preview SCADA
df_scada.describe()

In [ ]:
df_scada.head()

In [ ]:
df_scada.columns

# Evaluate Offsets (Wake Steering Case)

Compare the targeted offsets against what was achieved (as measured by the vane on the steering turbine SMV6 (index=5))

In [ ]:
# Specify offsets
start_of_offset = 200 # deg
end_of_offset = 240 # deg

In [ ]:
# Add a rounded ws column
df_scada = df_scada.assign(
    ws_round = df_scada.ws_005.round(),
    wd_to_plot = df_scada.wd_005
 )

# Limit to a few wind speeds for plotting
wind_speeds_to_plot = [6,8,10,12]
df_plot = df_scada[df_scada.ws_round.isin(wind_speeds_to_plot)]

# Melt together targeted and achieved 
df_plot = (df_plot
    [['wd_to_plot','ws_round','target_yaw_offset_005','wind_vane_005','control_mode']]
    # .melt(id_vars=['wd_round','ws_round','control_mode'],
    #       var_name='offset_type',
    #       value_name='offset_value')
    .sort_values(['wd_to_plot','ws_round'])
)

# Set up binning plots
x_edges = np.arange(start_of_offset - 20, end_of_offset + 20, 2)

fig, axarr = plt.subplots(len(wind_speeds_to_plot),2, figsize=(10,10))
print(wind_speeds_to_plot)
for ws_idx, ws in enumerate(wind_speeds_to_plot):
    print(f"wind speeds: {ws_idx}, {ws}")
    for c_idx, control_mode in enumerate(['baseline','controlled']):
        print(f"control mode: {c_idx}, {control_mode}")
        ax = axarr[ws_idx, c_idx]
        print(f"ax: {ax}")
        df_sub = df_plot[(df_plot.ws_round==ws) & (df_plot.control_mode==control_mode)]

        plot_binned_mean_and_ci(df_sub.wd_to_plot,
                                df_sub.target_yaw_offset_005,
                                color='k',
                                x_edges=x_edges,
                                label='Targetted Offset',
                                ax=ax)
        
        plot_binned_mean_and_ci(df_sub.wd_to_plot,
                                df_sub.wind_vane_005,
                                color='r',
                                x_edges=x_edges,
                                label='SMV6 Vane Measurement',
                                ax=ax)
        
        ax.set_xlim([start_of_offset-20,end_of_offset+20])
        ax.set_ylim([-30, 50])
        ax.grid(True)
        ax.axvline(start_of_offset,color='k',ls='--')
        ax.axvline(end_of_offset,color='k',ls='--')

        if ws_idx == 0:
            ax.set_title(f'Control Mode: {control_mode}')

        if c_idx == 0:
            ax.set_ylabel(f'Wind Speed: {ws} m/s\nYaw Misalignment (deg)')
        else:
            ax.set_ylabel('Yaw Misalignment (deg)')

        if (ws_idx == 0) and (c_idx == 0):
            ax.legend()


In [ ]:
# Limit SCADA to this region
df_scada = df_scada[(df_scada.wd_smarteole > (start_of_offset - 20)) &
                    (df_scada.wd_smarteole < (end_of_offset + 20))]

## Prepare SCADA for Computing Energy Ratios (Baseline + Wake Steering Cases)

The energy ratio class as presently implemented requires explicit identification of the dataframe of the reference wind direction, wind speed, and power columns: "wd," "ws," and "pow_ref." Here, these will be set equal to the reference variables used in the SMARTEOLE wake steering experiment, which was computed in "02_download_and_format_dataset.ipynb".

In [ ]:
# Assign wd, ws and pow ref and subset SCADA based on reference variables used in the SMARTEOLE wake steering experiment (TODO reference the experiment)
df_scada = (df_scada
    .assign(
        wd = lambda df_: df_['wd_smarteole'],
        ws = lambda df_: df_['ws_smarteole'],
        pow_ref = lambda df_: df_['pow_ref_smarteole']
    )
)

In [ ]:
# Split SCADA into baseline and wake steeering (controlled)
df_scada_baseline = df_scada[df_scada.control_mode=='baseline']
df_scada_controlled = df_scada[df_scada.control_mode=='controlled']

## Load FLORIS model

Specify the path of the Empirical Gaussian FLORIS Model (emgauss) YAML file. Instantiate the FLORIS model using this file. 

In [ ]:
# Specify emgauss model
wake_model = 'emgauss'

# Specify emgauss yaml file
yaml = os.path.join('floris_input', 'emgauss.yaml')

# Instantiate FLORIS model object using emgauss yaml
fi = FlorisInterface(yaml)

# Define D
D = fi.floris.farm.rotor_diameters[0]

## Prepare FLORIS for Tuning (Baseline Case)

Using the FlorisTuner class, generate and inspect the associated dataframe for FLORIS that will be used for the energy ratio comparision with SCADA. 

In [ ]:
# Instantiate a FLORIS model tuner object
floris_tuner_baseline = FlorisTuner(fi=fi,
                                    df_scada=df_scada_baseline, 
                                    num_turbines=7, 
                                    test_turbines=[4]) 

## Tune "wake_expansion_rates" Using the Interpolation/Mathematical Optimization Technique 

The parameter of interest in the baseline case is "wake_expansion_rates". For now, no breakpoints will be assumed and thus only the 1st wake expansion rate (wake_expansion_rates[0]) will be tuned.

This tuning process is comprised of the following steps:
1. Specify a discrete range of values (initial YAML to N) to set wake_expansion_rates[0] to.

2. Generate the FLORIS energy ratios for this range of wake_expansion_rates[0] and compare to the SCADA energy ratios.

3. Calculate the error (mean squared error) between the FLORIS and SCADA energy ratios.

4. Interpolate a function to represent the error curve. 

5. Solve the optimization problem of minimizing the error curve.

6. Set wake_expansion_rates[0] equal to the x value of the solution to the optimization problem.

In [ ]:
# Specify a range of wake expansion rates (assuming no breakpoints) values
wake_expansion_rates = np.linspace(start=0.01, 
                                   stop=0.2)

## Visualize Results

Plot the true error, predicted error (error curve), local minimum of the true error and the minimum determined by the optimization solution

In [ ]:
# Determine the optimal value for wake expansion rates
fi_baseline_tuned, optimal_wake_expansion_rate, optimal_baseline_err, baseline_err_curve, true_baseline_err = floris_tuner_baseline.tune_floris(case='baseline',
                                                                                                                             pow_ref_columns=[0, 1, 2, 6],
                                                                                                                             param_name='Wake Expansion Rates',
                                                                                                                             param_values=wake_expansion_rates,
                                                                                                                             wd_step=2.0,
                                                                                                                             ws_step=1.0,
                                                                                                                             wd_bin_width=2.0,
                                                                                                                             verbose=False,
                                                                                                                             plot_err=True,
                                                                                                                             plot_energy_ratios=True)  

## Prepare FLORIS for Tuning (Wake Steering Case)

Using the FlorisTuner class, generate and inspect the associated dataframe for FLORIS that will be used for the energy ratio comparision with SCADA. 

In [ ]:
# Instantiate a FLORIS model tuner object
floris_tuner_controlled = FlorisTuner(fi=fi,
                                      df_scada=df_scada_controlled, 
                                      num_turbines=7, 
                                      test_turbines=[5],
                                      steered_turbine=5,
                                      yaw=df_scada_controlled.target_yaw_offset_005.values) 

## Tune "horizontal_deflection_gain_D" Using the Interpolation/Mathematical Optimization Technique 

The parameter of interest in the wake steering case is "horizontal_deflection_gain_D".

This tuning process is comprised of the following steps:

1. Specify a discrete range of values (initial YAML to N) to set horizontal_deflection_gain_D to.

2. Generate the FLORIS energy ratios for this range of horizontal_deflection_gain_D and compare to the SCADA energy ratios.

3. Calculate the error (mean squared error) between the FLORIS and SCADA energy ratios.

4. Interpolate a function to represent the error curve. 

5. Solve the optimization problem of minimizing the error curve.

6. Set horizontal_deflection_gain_D equal to the x value of the solution to the optimization problem.

In [ ]:
# Specify a range of horizontal deflection values
horizontal_deflection_gain_D = np.linspace(start=0.0, 
                                           stop=2.0)

## Visualize Results

Plot the true error, predicted error (error curve), local minimum of the true error and the minimum determined by the optimization solution

In [ ]:
# Determine the optimal value for horizontal deflection gain
fi_controlled_tuned, optimal_horizontal_deflection_gain_D, optimal_controlled_err, controlled_err_curve, true_controlled_err = floris_tuner_controlled.tune_floris(case='controlled',
                                                                                                                                                                   pow_ref_columns=[0, 1, 2, 6],
                                                                                                                                                                   param_name='Horizontal Deflection Gain D',
                                                                                                                                                                   param_values=horizontal_deflection_gain_D,
                                                                                                                                                                   wd_step=2.0,
                                                                                                                                                                   ws_step=1.0,
                                                                                                                                                                   wd_bin_width=2.0,
                                                                                                                                                                   verbose=False,
                                                                                                                                                                   plot_err=True,
                                                                                                                                                                   plot_energy_ratios=True)  

## Save Tuned FLORIS Parameters to a YAML File

 Output the parameters of the tuned FLORIS models to YAML files for future reference

In [ ]:
# Specify file paths to write the YAML files to
baseline_filepath = os.path.join(Path.cwd(), 'floris_input', 'tuned_baseline_emgauss.yaml')
controlled_filepath = os.path.join(Path.cwd(), 'floris_input', 'tuned_controlled_emgauss.yaml')

In [ ]:
# Write the YAML files
floris_tuner_baseline.write_yaml(baseline_filepath)
floris_tuner_controlled.write_yaml(controlled_filepath)